## Install Required Packages

In [ ]:
sudo apt -y install build-essential libncurses-dev bison flex libssl-dev libelf-dev

In [ ]:
pip install tensorflow==1.15.2
pip install keras==2.1.3
pip install pandas
pip install scikit-learn
pip install numpy
pip install pyprimo

In [ ]:
lsblk

## STEP 1: Collect Trace and Train the LinnOS ML model 

In [ ]:
# Several minutes to several hours.
TraceTag='trace'
cd LinnOSWriterReplayer
nohup sudo ./writer /dev/sdb 'testTraces/anonymous.drive0.'$TraceTag &
nohup sudo ./writer /dev/sdc 'testTraces/anonymous.drive1.'$TraceTag &
id_1=$(sudo pgrep -a writer | awk 'NR==1 {print $1}')
id_2=$(sudo pgrep -a writer | awk 'NR==2 {print $1}')
sudo ./writer /dev/sdd 'testTraces/anonymous.drive2.'$TraceTag
wait $id_1
wait $id_2
cd ..

In [ ]:
# 10 minutes
cd LinnOSWriterReplayer
sudo ./replayer_fail /dev/sdb-/dev/sdc-/dev/sdd \
'testTraces/traindrive0.'$TraceTag \
'testTraces/traindrive1.'$TraceTag \
'testTraces/traindrive2.'$TraceTag py/TrainTraceOutput

In [ ]:
cd ..
for i in 0 1 2 
do
   python3 LinnOSWriterReplayer/py/traceParser.py direct 3 4 \
   LinnOSWriterReplayer/py/TrainTraceOutput LinnOSWriterReplayer/mlData/temp1 \
   LinnOSWriterReplayer/mlData/"mldrive${i}.csv" "$i"
done

In [ ]:
# 10~20 minutes
for i in 0 1 2 
do
   python3 LinnOSWriterReplayer/py/pred1.py \
   LinnOSWriterReplayer/mlData/"mldrive${i}.csv" > "mldrive${i}results".txt
done

In [ ]:
cd LinnOSWriterReplayer/mlData
mkdir -p drive0weights
mkdir -p drive1weights
mkdir -p drive2weights
cp mldrive0.csv.* drive0weights
cp mldrive1.csv.* drive1weights
cp mldrive2.csv.* drive2weights
cd ../..

In [ ]:
# Replace with Trained New model

python3 LinnOSWriterReplayer/mlHeaderGen/mlHeaderGen.py \
Trace sdb ~/LinnOS/LinnOSWriterReplayer/mlData/drive0weights ~/LinnOS/linux-5.4.8-linnos/block

python3 LinnOSWriterReplayer/mlHeaderGen/mlHeaderGen.py \
Trace sdc ~/LinnOS/LinnOSWriterReplayer/mlData/drive1weights ~/LinnOS/linux-5.4.8-linnos/block

python3 LinnOSWriterReplayer/mlHeaderGen/mlHeaderGen.py \
Trace sdd ~/LinnOS/LinnOSWriterReplayer/mlData/drive2weights ~/LinnOS/linux-5.4.8-linnos/block

## STEP 2: Install Retrained LinnOS kernel

**Important**: To avoid install two kernel with same name, you need to rename the `EXTRAVERSION = -linnos` to `EXTRAVERSION = -linnosRe` in `/linux-5.4.8-linnos/Makefile`

In [ ]:
# IF ERROR: make[1]: *** No rule to make target 'debian/canonical-certs.pem', needed by 'certs/x509_certificate_list'.  Stop.
# SOLVE:  in .config  CONFIG_SYSTEM_TRUSTED_KEYS="debian/canonical-certs.pem"  --> CONFIG_SYSTEM_TRUSTED_KEYS=""

In [ ]:
sudo make -C ~/LinnOS/linux-5.4.8-linnos distclean
make -C ~/LinnOS/linux-5.4.8-linnos olddefconfig

In [ ]:
# 20 minutes
make -C ~/LinnOS/linux-5.4.8-linnos -j $(nproc) > makeLinnosLog.txt

In [ ]:
sudo make -C ~/LinnOS/linux-5.4.8-linnos modules_install > modulesInstallLinnosLog.txt

In [ ]:
sudo make -C ~/LinnOS/linux-5.4.8-linnos install 

In [ ]:
sudo update-initramfs -c -k 5.4.8

In [ ]:
sudo update-grub

Please refer to `Change Kernel Version` section (bottom) of this note book to change the kernel version

## STEP 3: Evaluate LinnOS

In [ ]:
TraceTag='trace'

In [ ]:
cd LinnOSWriterReplayer
sudo ./replayer_fail /dev/sdb-/dev/sdc-/dev/sdd \
'testTraces/testdrive0.'$TraceTag \
'testTraces/testdrive1.'$TraceTag \
'testTraces/testdrive2.'$TraceTag py/MLOutput1

In [ ]:
cd ..
python3 LinnOSWriterReplayer/py/percentile.py 2 read \
LinnOSWriterReplayer/py/MLOutput1 LinnOSWriterReplayer/py/MLData1

## STEP 4: Obtain Primo Model

In [ ]:
pip install sklearn-porter 
# No module named 'sklearn.tree.tree'
# Version may mismatch, you can build a new conda env to convert tree into C format

In [ ]:
bash run_parser.sh

In [ ]:
python primo_model.py -d 0
python primo_model.py -d 1
python primo_model.py -d 2

In [ ]:
# Need to switch to another env to enable sklearn-porter 
python portC.py -d 0
python portC.py -d 1
python portC.py -d 2

Now you obtain the weights of PrDT for each drive in C format (such as `./traces/primodrive0.c`). 

Next, you can refer to `./linux-5.4.8-primo/block/Trace/w_sdb.h` to manually replace the weight value for each drive.

Note `convert_thresholds.c` is used to convert threshold (e.g.,`double thresholds_sdb[13]`) into integer format (e.g., `int ithresholds_sdb[13]`). Linux kernel not support float value.

## STEP 5: Install ReTrained Primo kernel

**Important**: To avoid install two kernel with same name, you need to rename the `EXTRAVERSION = -primo` to `EXTRAVERSION = -primoRe` in `/linux-5.4.8-primo/Makefile`

In [ ]:
sudo apt -y install build-essential libncurses-dev bison flex libssl-dev libelf-dev

In [ ]:
sudo make -C ~/LinnOS/linux-5.4.8-primo distclean
make -C ~/LinnOS/linux-5.4.8-primo olddefconfig

In [ ]:
# 20 minutes
make -C ~/LinnOS/linux-5.4.8-primo -j $(nproc) > makePrimoLog.txt

In [ ]:
sudo make -C ~/LinnOS/linux-5.4.8-primo modules_install > modulesInstallPrimoLog.txt

In [ ]:
sudo make -C ~/LinnOS/linux-5.4.8-primo install 

In [ ]:
sudo update-initramfs -c -k 5.4.8

In [ ]:
sudo update-grub

Please refer to `Change Kernel Version` section (bottom) of this note book to change the kernel version

## STEP 6: Evaluate Primo

In [ ]:
TraceTag='trace'
cd LinnOSWriterReplayer
sudo ./replayer_fail /dev/sdb-/dev/sdc-/dev/sdd \
'testTraces/testdrive0.'$TraceTag \
'testTraces/testdrive1.'$TraceTag \
'testTraces/testdrive2.'$TraceTag py/PrimoOutput1

In [ ]:
cd ..
python3 LinnOSWriterReplayer/py/percentile.py 2 read \
LinnOSWriterReplayer/py/PrimoOutput1 LinnOSWriterReplayer/py/PrimoData1

## Change Kernel Version

In [ ]:
awk -F\' '$1=="menuentry " || $1=="submenu " {print i++ " : " $2}; /\tmenuentry / {print "\t" i-1">"j++ " : " $2};' /boot/grub/grub.cfg

In [ ]:
#### **Important**: Edit /etc/default/grub outside (Not within this notebook) to change the current kernel to Primo

> sudo nano /etc/default/grub      

  Change the First line `GRUB_DEFAULT=0` to `GRUB_DEFAULT="1>0"`   (Refer to the above output, the Primo kenerl may not be "1>0". Typically, it display 1>0 : Ubuntu, with Linux 5.4.8-primo)
  
> sudo update-grub

> sudo reboot